## 1. Set environment vairable for MemoryDB cluster 

In [1]:
%env MEMORYDB_HOST=semanticcache.ghlaqp.clustercfg.memorydb.us-east-1.amazonaws.com
%env MEMORYDB_PORT=6379

env: MEMORYDB_HOST=semanticcache.ghlaqp.clustercfg.memorydb.us-east-1.amazonaws.com
env: MEMORYDB_PORT=6379


## 2. Install packages

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langchain_core
!{sys.executable} -m pip install langchain_aws
!{sys.executable} -m pip install redis

In [2]:
import os
from langchain_core.globals import set_llm_cache
from langchain_aws.cache import InMemorySemanticCache
from langchain_aws import ChatBedrock
from langchain_aws.embeddings import BedrockEmbeddings
import redis
from redis.cluster import RedisCluster as MemoryDB

## Initialize the ChatBedrock and embeddings 

In [3]:
# create the Anthropic Model
model_kwargs = {
    "temperature": 0, 
    "top_k": 250, 
    "top_p": 1,
    "stop_sequences": ["\\n\\nHuman:"]
}    

In [4]:
# use the Anthropic Claude model
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=model_kwargs
)


In [5]:
# create a Titan Embeddings client
embeddings = BedrockEmbeddings()

## Connect to MemoryDB

In [6]:
%%time
memorydb_host = os.environ.get("MEMORYDB_HOST", "localhost")
memorydb_port = os.environ.get("MEMORYDB_PORT", 6379)
# print(f"MemoryDB Url = {memorydb_host}:{memorydb_port}")
rc = MemoryDB(host=memorydb_host, port=memorydb_port, ssl=False, decode_responses=False, ssl_cert_reqs="none")
rc.ping()
#rc.flushall()

CPU times: user 60.8 ms, sys: 0 ns, total: 60.8 ms
Wall time: 67.2 ms


True

## Ask a question without a cache

In [7]:
%%time
response=llm.invoke("Tell me about mission to moon")
print(response.content)

Here are some key details about missions to the Moon:

- The first human missions to the Moon were the crewed missions of the United States Apollo program between 1968-1972. A total of 12 astronauts walked on the lunar surface during the six successful NASA landing missions.

- On July 20, 1969, Apollo 11 marked the first crewed landing on the Moon. Neil Armstrong became the first person to walk on the lunar surface, famously saying "That's one small step for man, one giant leap for mankind."

- The last crewed landing was Apollo 17 in December 1972. Since then, numerous uncrewed spacecraft have explored the Moon from orbit.

- Some key uncrewed lunar missions include the Soviet Luna program, NASA's Lunar Reconnaissance Orbiter mapping the Moon since 2009, and recent missions like China's Chang'e 4 which landed on the far side of the Moon in 2019.

- NASA is currently working towards the Artemis program which aims to land the first woman and next man on the lunar surface by 2024 and es

## Enable MemoryDB for durable semantic caching 

In [8]:
set_llm_cache(
    InMemorySemanticCache(redis_url=f"redis://{memorydb_host}:{memorydb_port}/ssl=True&ssl_cert_reqs=none",
                embedding=embeddings)
)

In [9]:
%%time
response=llm.invoke("Tell me about mission to moon")
print(response.content)

Here are some key details about missions to the Moon:

- The first human missions to the Moon were the crewed missions of the United States Apollo program between 1968-1972. A total of 12 astronauts walked on the lunar surface during the six successful NASA landing missions.

- On July 20, 1969, Apollo 11 marked the first crewed landing on the Moon. Neil Armstrong became the first person to walk on the lunar surface, famously saying "That's one small step for man, one giant leap for mankind."

- The last crewed landing was Apollo 17 in December 1972. Since then, numerous uncrewed spacecraft have explored the Moon from orbit.

- Some key robotic lunar missions include the Soviet Luna program, NASA's Lunar Reconnaissance Orbiter and Lunar Crater Observation and Sensing Satellite, India's Chandrayaan-1, and China's Chang'e program which landed rovers on the Moon.

- NASA's Artemis program aims to land the first woman and next man on the lunar surface by 2024 and establish sustainable luna

In [10]:
%%time
response=llm.invoke("Who first invented a telescope")
print(response.content)

The telescope was first invented in the Netherlands in the early 17th century, around 1608. However, there is some debate over who should get credit as the original inventor.

The main contenders are:

1) Hans Lippershey - A Dutch eyeglass maker who is often credited as the first to apply for a patent for his refracting telescope in 1608, though his patent request was denied.

2) Zacharias Janssen - Another Dutch spectacle maker who may have invented a compound microscope and telescope around 1590, though the exact dates are uncertain.

3) Jacob Metius - A Dutch professor and maker of optical instruments who claimed to have invented the telescope in 1608, around the same time as Lippershey.

While there is no consensus on who was truly first, most historians give credit to Hans Lippershey as the first person known to have successfully applied for a patent for the refracting telescope in 1608. Galileo Galilei later made improvements and was one of the first to use the telescope for astr

In [11]:
%%time
response=llm.invoke("Who first invented a car")
print(response.content)

The invention of the modern car is credited to several pioneers who contributed different components and concepts over time. However, some key figures stand out:

1. Karl Benz (1844-1929) - A German engineer who patented the first gasoline-powered automobile in 1886. His three-wheeled Benz Patent Motorwagen is widely regarded as the first modern automobile.

2. Gottlieb Daimler (1834-1900) - Another German engineer who independently designed and built a high-speed petrol engine in 1885, which he fitted to a bicycle, creating the first motorcycle. He later collaborated with Wilhelm Maybach to build the first four-wheeled automobile in 1889.

3. Henry Ford (1863-1947) - An American engineer and industrialist who didn't invent the automobile itself, but revolutionized its production with the introduction of the assembly line and mass production techniques. His Model T, introduced in 1908, made cars affordable for the masses.

4. Nicolas Joseph Cugnot (1725-1804) - A French engineer who bu

In [12]:
%%time
respone3=llm.invoke("Who  first  a Telescope")
print(respone3.content)

The telescope was first invented in the Netherlands in the early 17th century, around 1608. However, there is some debate over who should get credit as the original inventor.

The main contenders are:

1) Hans Lippershey - A Dutch eyeglass maker who is often credited as the first to apply for a patent for his refracting telescope in 1608, though his patent request was denied.

2) Zacharias Janssen - Another Dutch spectacle maker who may have invented a compound microscope and telescope around 1590, though the exact dates are uncertain.

3) Jacob Metius - A Dutch professor and maker of optical instruments who claimed to have invented the telescope in 1608, around the same time as Lippershey.

While there is no consensus on who was truly first, most historians give credit to Hans Lippershey as the first person known to have successfully applied for a patent for the refracting telescope in 1608. Galileo Galilei later made improvements and was one of the first to use the telescope for astr